In [1]:
import pandas as pd

**TODO: Replace special characters within the data files**

**TODO: Check wheter training the algorithms on just one league improves performance**

In [2]:
li = []

li.append(pd.read_csv("../data/premier_league_2023-2023.csv", index_col=0))
# li.append(pd.read_csv("../data/la_liga_2019-2023.csv", index_col=0))
# li.append(pd.read_csv("../data/bundesliga_2019-2023.csv", index_col=0))
# li.append(pd.read_csv("../data/serie_a_2019-2023.csv", index_col=0))

matches = pd.concat(li, axis=0, ignore_index=True)
matches.shape

(436, 58)

In [3]:
del matches["notes"]
del matches["match report"]

In [4]:
matches["target"] = (matches["result"] == "W").astype("int")

In [5]:
matches["team_code"] = matches["team"].astype("category").cat.codes
matches["opponent_code"] = matches["opponent"].astype("category").cat.codes

In [6]:
def rolling_averages(group):
    group = group.sort_values("date") # date wird immer spaeter von 2019 bis 2024

    # Performance
    rolling_stats = group["gf"].rolling(5, closed='left').mean()
    group["gf_rolling"] = rolling_stats

    rolling_stats = group["ga"].rolling(5, closed='left').mean()
    group["ga_rolling"] = rolling_stats

    rolling_stats = group["sh"].rolling(3, closed='left').mean()
    group["sh_rolling"] = rolling_stats

    rolling_stats = group["sot%"].rolling(3, closed='left').mean()
    group["sot%_rolling"] = rolling_stats

    rolling_stats = group["dist"].rolling(3, closed='left').mean()
    group["dist_rolling"] = rolling_stats


    # Goalkeeper
    rolling_stats = group["sota"].rolling(2, closed='left').mean()
    group["sota_rolling"] = rolling_stats

    rolling_stats = group["ga"].rolling(2, closed='left').mean()
    group["ga_rolling"] = rolling_stats

    rolling_stats = group["opp"].rolling(2, closed='left').mean()
    group["opp_rolling"] = rolling_stats

    rolling_stats = group["stp"].rolling(2, closed='left').mean()
    group["stp_rolling"] = rolling_stats

    group = group.dropna(subset=["gf_rolling", "ga_rolling", "sh_rolling", "sot%_rolling", "dist_rolling", "sota_rolling", "ga_rolling", "opp_rolling", "stp_rolling"])

    return group

In [7]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "sota"]
new_cols = [f"{c}_rolling" for c in cols]

In [8]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x))
matches_rolling = matches_rolling.droplevel('team')

/var/folders/yn/th9j1n395vq70h3nx8132vg00000gn/T/ipykernel_70711/323149295.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x))


In [9]:
matches_rolling.index = range(matches_rolling.shape[0])

In [10]:
for col in cols:
	del matches_rolling[col]

In [11]:
def write_list_into_csv(match_df, name: str):
	match_df.columns = [c.lower() for c in match_df.columns]
	match_df.to_csv(name)

In [12]:
write_list_into_csv(matches_rolling, "../prepared_data/test.csv")

In [14]:
matches_rolling[matches_rolling["date"] > '2024-01-30']

,date,time,round,day,venue,result,opponent,xg,xga,poss,...,team_code,opponent_code,gf_rolling,ga_rolling,sh_rolling,sot%_rolling,dist_rolling,sota_rolling,opp_rolling,stp_rolling
49,2024-02-01,19:30,Matchweek 22,Thu,Away,D,West Ham,1.2,1.4,52.0,...,2,18,2.0,3.5,16.666667,15.266667,18.500000,6.5,12.5,0.0
65,2024-01-31,19:30,Matchweek 22,Wed,Away,L,Tottenham,1.6,2.9,32.0,...,3,17,1.2,2.5,11.333333,39.500000,14.866667,4.5,16.5,2.0
99,2024-01-31,19:30,Matchweek 22,Wed,Away,L,Manchester City,1.4,1.7,26.0,...,5,12,1.0,2.0,10.333333,36.466667,16.966667,6.0,16.0,2.5
116,2024-01-31,20:15,Matchweek 22,Wed,Away,L,Liverpool,0.6,2.9,50.0,...,6,10,1.8,1.0,12.000000,34.733333,16.766667,5.0,21.0,3.0
184,2024-01-31,20:15,Matchweek 22,Wed,Home,W,Chelsea,2.9,0.6,50.0,...,10,6,2.2,1.0,21.666667,46.700000,17.600000,2.0,10.5,0.5
216,2024-01-31,19:30,Matchweek 22,Wed,Home,W,Burnley,1.7,1.4,74.0,...,12,5,2.4,1.0,22.333333,33.100000,18.733333,3.5,7.0,0.5
233,2024-02-01,20:15,Matchweek 22,Thu,Away,W,Wolves,3.3,2.1,47.0,...,13,19,1.2,2.0,10.666667,42.000000,16.666667,3.5,20.0,1.0
301,2024-01-31,19:30,Matchweek 22,Wed,Home,W,Brentford,2.9,1.6,68.0,...,17,3,2.2,1.5,15.666667,32.366667,14.766667,3.0,20.5,1.0
318,2024-02-01,19:30,Matchweek 22,Thu,Home,D,Bournemouth,1.4,1.2,48.0,...,18,2,1.8,1.0,8.666667,33.333333,16.100000,7.0,10.5,1.5
335,2024-02-01,20:15,Matchweek 22,Thu,Home,L,Manchester Utd,2.1,3.3,53.0,...,19,13,1.8,0.0,10.333333,44.333333,16.600000,1.5,17.0,0.0
